In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from matplotlib import pyplot as plt
from tensorflow.python.client import device_lib
import tensorflow as tf

In [2]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout


In [3]:
import cv2
import imghdr

In [4]:
gpus = tf.config.experimental.list_physical_devices("GPU")
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
main_directory = r'H:\sprints\face recognition task\105_classes_pins_dataset' 

In [7]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Specify the path to the main directory
main_directory = r'H:\sprints\face recognition task\105_classes_pins_dataset' 

# Check if the main directory exists
if not os.path.exists(main_directory):
    print("The specified main directory does not exist.")
else:
    print("Main directory exists, proceeding...")

# Initialize counts
folder_count = 0
total_file_count = 0

# Iterate through the main directory
for root, dirs, files in os.walk(main_directory):
    # Count folders
    folder_count += len(dirs)
    
    # Count files in each folder
    folder_file_count = len(files)
    total_file_count += folder_file_count
    
    # Print the folder name and the number of files in that folder
    print(f"Folder: {os.path.basename(root)}, Files: {folder_file_count}")
    

# Print the overall results
print(f"\nNumber of folders: {folder_count}")
print(f"Total number of files: {total_file_count}")


In [8]:
# Specify the paths
main_directory = r'H:\\sprints\\face recognition task\\105_classes_pins_dataset'  
output_folder = r'H:\\sprints\\face recognition task\\cropped_faces'  


In [9]:
import cv2
import os

# Specify the paths
main_directory = r'H:\\sprints\\face recognition task\\105_classes_pins_dataset'  
output_folder = r'H:\\sprints\\face recognition task\\cropped_faces'  

# Target size for output images
target_size = (128, 128)

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Iterate through each class folder
for class_folder in os.listdir(main_directory):
    class_path = os.path.join(main_directory, class_folder)

    if os.path.isdir(class_path):  # Ensure it is a directory
        # Create a specific output subfolder for each class
        class_output_folder = os.path.join(output_folder, class_folder)
        if not os.path.exists(class_output_folder):
            os.makedirs(class_output_folder)  # Create subfolder for class if it doesn't exist
        
        for img_name in os.listdir(class_path):
            if img_name.endswith(('.jpg', '.png', '.jpeg')):
                # Read the image
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                
                if img is None:  # Skip if image is not readable
                    print(f"Failed to read image: {img_path}")
                    continue
                
                # Convert the image to grayscale for detection
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Detect faces
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

                # Crop and save each detected face
                for i, (x, y, w, h) in enumerate(faces):
                    face = img[y:y+h, x:x+w]
                    
                    # Resize the cropped face to the target size
                    resized_face = cv2.resize(face, target_size)
                    
                    # Generate a unique filename
                    face_filename = f"{img_name.split('.')[0]}_face_{i}.jpg"
                    face_path = os.path.join(class_output_folder, face_filename)
                    
                    # Save the resized cropped face
                    cv2.imwrite(face_path, resized_face)

                print(f"Processed image: {img_name} in class '{class_folder}', Found {len(faces)} faces.")

print("Face detection, cropping, and resizing completed.")


In [10]:
import os
from tensorflow.keras.preprocessing.image import load_img
dataset = []
count=0
for file in os.listdir(output_folder):
    path=os.path.join(output_folder,file)
    t=0
    for img in os.listdir(path):
        image=load_img(os.path.join(path,img),grayscale=False,color_mode='rgb',target_size=(128,128))
        image = np.array(image)/255.0
        t+=1
        dataset.append([image,count])
    count+=1

In [11]:
from tensorflow.keras.utils import to_categorical 
data = []
labels = []

for image, label in dataset:
    data.append(image)
    labels.append(label)

labels_1 = to_categorical(labels)
data=np.array(data)
labels1=np.array(labels_1)

In [12]:
# print(data)
# print(labels1)

In [13]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(data,labels1,test_size=0.2,random_state=44)


In [15]:
from tensorflow.keras.applications.vgg16 import VGG16


# Load VGG16 model with pre-trained weights, excluding the top (fully connected layers)
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# # Freeze all layers in the base VGG16 model
# base_model.trainable = False
# for layer in base_model.layers[-1:]:
#     layer.trainable = True

# Load the VGG model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the layers of the VGG model
for layer in base_model.layers:
    layer.trainable = False
# import os

# weights_path = r'H:\sprints\face recognition task\resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# if os.path.exists(weights_path):
#     from tensorflow.keras.applications import ResNet50

#     # Load ResNet50 model with custom weights, excluding the fully connected layers (include_top=False)
#     base_model = ResNet50(weights=weights_path, include_top=False, input_shape=(224, 224, 3))

#     # Freeze the layers of the ResNet model
#     for layer in base_model.layers:
#         layer.trainable = False

#     base_model.summary()
# else:
#     print(f"Weights file not found at: {weights_path}")


In [16]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [17]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout

# Create a new model with additional modifications
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.0001)),  
    BatchNormalization(),
    Dropout(0.2), 
    Dense(512, activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(105, activation='softmax') 
])


In [18]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Set up callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
# lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)


# Train the model
history = model.fit(
    x_train,
    y_train,
    epochs=70,
    batch_size=12,
    validation_data=(x_test,y_test),
    callbacks=[early_stopping],
)

Epoch 1/70
1026/1026 [==============================] - 26s 20ms/step - loss: 4.4006 - accuracy: 0.0763 - val_loss: 3.6049 - val_accuracy: 0.1764
Epoch 2/70
1026/1026 [==============================] - 20s 19ms/step - loss: 3.7148 - accuracy: 0.1729 - val_loss: 3.4511 - val_accuracy: 0.2277
Epoch 3/70
1026/1026 [==============================] - 19s 19ms/step - loss: 3.4238 - accuracy: 0.2520 - val_loss: 3.3726 - val_accuracy: 0.2793
Epoch 4/70
1026/1026 [==============================] - 19s 19ms/step - loss: 3.2173 - accuracy: 0.3118 - val_loss: 3.2608 - val_accuracy: 0.3105
Epoch 5/70
1026/1026 [==============================] - 20s 19ms/step - loss: 3.0566 - accuracy: 0.3551 - val_loss: 3.1912 - val_accuracy: 0.3501
Epoch 6/70
1026/1026 [==============================] - 19s 19ms/step - loss: 2.9402 - accuracy: 0.3948 - val_loss: 3.0710 - val_accuracy: 0.3771
Epoch 7/70
1026/1026 [==============================] - 19s 19ms/step - loss: 2.8359 - accuracy: 0.4215 - val_loss: 3.1310 -

In [14]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(12312, 128, 128, 3) (12312, 105)
(3079, 128, 128, 3) (3079, 105)


In [15]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import InceptionV3

weights = 'H:\sprints\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5' 
model_in = InceptionV3(input_shape =(128,128,3),include_top=False,weights=None)
model_in.load_weights(weights)
for layer in model_in.layers:
    layer.trainable = False
    

In [16]:
model_in.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 63, 63, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 63, 63, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [17]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
# history = Sequential([
#     model_in,
#     Flatten(),
# ])

In [18]:
last_layer = model_in.get_layer('mixed5')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output


last layer output shape: (None, 6, 6, 768)


In [19]:
from tensorflow.keras import layers

x = layers.Flatten()(last_output)
print(x.shape)


(None, 27648)


In [20]:
x = layers.Dense(1024, activation='leaky_relu', activity_regularizer=l2(0.0001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

In [21]:
x = layers.Dense(512, activation='leaky_relu', activity_regularizer=l2(0.0001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

In [22]:
# Final output layer
x = layers.Dense(105, activation='softmax')(x)

In [23]:
from tensorflow.keras.models import Model

model=Model(model_in.input,x)

In [24]:
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam,RMSprop
model.compile(optimizer=Adam(learning_rate=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
# model.compile(optimizer=RMSprop(learning_rate=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
batch_size=32
epochs=100

early_stopping = EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True)


In [26]:
history = model.fit(x_train,y_train,batch_size=batch_size,epochs=100,validation_data=[x_test,y_test],callbacks=[early_stopping])

Epoch 1/100
385/385 [==============================] - 12s 22ms/step - loss: 5.5980 - accuracy: 0.0292 - val_loss: 3.8551 - val_accuracy: 0.1650
Epoch 2/100
385/385 [==============================] - 7s 18ms/step - loss: 4.4453 - accuracy: 0.1052 - val_loss: 3.1946 - val_accuracy: 0.3144
Epoch 3/100
385/385 [==============================] - 7s 19ms/step - loss: 3.7634 - accuracy: 0.1898 - val_loss: 2.7708 - val_accuracy: 0.4167
Epoch 4/100
385/385 [==============================] - 7s 18ms/step - loss: 3.2118 - accuracy: 0.2783 - val_loss: 2.4663 - val_accuracy: 0.4826
Epoch 5/100
385/385 [==============================] - 7s 17ms/step - loss: 2.8115 - accuracy: 0.3560 - val_loss: 2.2227 - val_accuracy: 0.5450
Epoch 6/100
385/385 [==============================] - 7s 17ms/step - loss: 2.4860 - accuracy: 0.4225 - val_loss: 2.0339 - val_accuracy: 0.5836
Epoch 7/100
385/385 [==============================] - 7s 18ms/step - loss: 2.2189 - accuracy: 0.4802 - val_loss: 1.8982 - val_accuracy

In [27]:
loss , accuracy = model.evaluate(x_test,y_test)

97/97 [==============================] - 1s 14ms/step - loss: 0.7421 - accuracy: 0.8496


In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the simplified data augmentation transformations
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=3,  # Random rotation between 0 and 10 degrees
)

# Validation data only requires rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Apply the simplified data augmentation to the training data
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

# Apply rescaling to the validation data
validation_generator = test_datagen.flow(x_test, y_test, batch_size=32)


In [31]:
history = model.fit(train_generator,batch_size=batch_size,epochs=70,validation_data=validation_generator,callbacks=[early_stopping])

Epoch 1/70
385/385 [==============================] - 22s 56ms/step - loss: 2.9275 - accuracy: 0.3242 - val_loss: 20.9588 - val_accuracy: 0.0451
Epoch 2/70
385/385 [==============================] - 23s 59ms/step - loss: 2.8639 - accuracy: 0.3338 - val_loss: 22.4688 - val_accuracy: 0.0169
Epoch 3/70
385/385 [==============================] - 20s 53ms/step - loss: 2.7910 - accuracy: 0.3492 - val_loss: 24.3655 - val_accuracy: 0.0188
Epoch 4/70
385/385 [==============================] - 21s 54ms/step - loss: 2.7433 - accuracy: 0.3581 - val_loss: 26.6010 - val_accuracy: 0.0240
Epoch 5/70
385/385 [==============================] - 21s 54ms/step - loss: 2.6827 - accuracy: 0.3648 - val_loss: 26.9618 - val_accuracy: 0.0114
Epoch 6/70
385/385 [==============================] - 21s 54ms/step - loss: 2.6637 - accuracy: 0.3713 - val_loss: 31.4449 - val_accuracy: 0.0192
